In [131]:
from datascience import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
import json

In [132]:
# 10/08 Note: research_fund_comb.csv is a complete csv of the initial pdf file 
combined_t = Table.read_table('data/research_fund_comb.csv')
combined_t.show(20)

Control Unit,Division,Department,Direct Sponsor,2014,2015,2016
Campus Support,Vice Chancellor - Administration,Finance,LBNL Lawrence Berkeley Nati..,"$75,000","$400,000",nan
Campus Support,Vice Chancellor - Administration,Finance,SAMHSA Center for Substanc..,$0,$0,nan
Campus Support,Vice Chancellor - Administration,Finance,UC Office of the President,$0,nan,nan
Campus Support,Vice Chancellor - Administration,Information Services & Technology,Andrew W Mellon Foundation,"$150,000",$0,nan
Campus Support,Vice Chancellor - Administration,Information Services & Technology,LYRASIS,nan,"$100,018","$70,854"
Campus Support,Vice Chancellor - Administration,Information Services & Technology,NSF National Science Founda..,nan,nan,"$399,446"
Campus Support,Vice Chancellor - Administration,International House,Middlebury College,nan,"$10,000",nan
Campus Support,Vice Chancellor - Administration,Public Safety,Bay Areas Urban Area Securit..,"$327,226","$30,145",nan
Campus Support,Vice Chancellor - Real Estate,Physical Plant Campus Services,California Emergency Manage..,nan,nan,"$573,828"
Colleges,CNR College of Natural Resources,Null,"Abt Associates, Inc.","$73,942",nan,nan


In [133]:
combined_t.where('Direct Sponsor', 'BP Group (BP America, BP E..')

Control Unit,Division,Department,Direct Sponsor,2014,2015,2016
Colleges,College of Chemistry,Chemical and Biomolecular Engineering,"BP Group (BP America, BP E..",nan,"$1,350,000",nan
Colleges,College of Engineering,ERSO Engineering Research Support Organization,"BP Group (BP America, BP E..","($200,000)",nan,nan
Vice Chancellor - Research,VC Res Other Research Units,EBI Energy Biosciences Institute,"BP Group (BP America, BP E..",nan,nan,"$5,000,000"
Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Biosciences (QB3),"BP Group (BP America, BP E..","$10,000",nan,nan
Vice Chancellor - Research,Vice Chancellor for Research ORUs,Null,"BP Group (BP America, BP E..",$0,$0,$0


In [134]:
# 10/09 Note: All sponsors downloaded from the Cal Phoebe site 
phoebe = Table.read_table("data/PhoebeSearchResults.csv")
phoebe

Sponsor Class,Sponsor Category,Sponsor Type,Sponsor Code,Sponsor,Sponsor Acronym
Federal,Other Federal Category,All Other Federal,150,CIA Central Intelligence Agency,CIA
Federal,Other Federal Category,All Other Federal,5510,Federal Mediation & Conciliation Service,FMCS
Federal,Other Federal Category,All Other Federal,5440,National Commission on Air Quality,nan
Federal,Other Federal Category,All Other Federal,5350,VA Department of Veterans Affairs,VA
Federal,Other Federal Category,All Other Federal,5325,Smithsonian - National Museum Natural History,nan
Federal,Other Federal Category,All Other Federal,5305,Smithsonian - Office of Fellowships & Grants,nan
Federal,Other Federal Category,All Other Federal,5302,Smithsonian - Office of Environmental Sciences,nan
Federal,Other Federal Category,All Other Federal,5301,Smithsonian Institution,nan
Federal,Other Federal Category,All Other Federal,5290,National Archives and Records Administration,nan
Federal,Other Federal Category,All Other Federal,5285,SBA Small Business Administration,SBA


In [135]:
phoebe.where('Sponsor', 'Stifterverband fur die Deutsch')

Sponsor Class,Sponsor Category,Sponsor Type,Sponsor Code,Sponsor,Sponsor Acronym
Not for Profit,Business Related Nonprofit Institutes and Associat,Foreign Trade Associations,4591,Stifterverband fur die Deutsch,nan


In [136]:
phoebe_trunc = phoebe.with_column("SponsorName_trunc", 
                                 phoebe.apply(lambda x: x[0:28], "Sponsor"))
phoebe_trunc

Sponsor Class,Sponsor Category,Sponsor Type,Sponsor Code,Sponsor,Sponsor Acronym,SponsorName_trunc
Federal,Other Federal Category,All Other Federal,150,CIA Central Intelligence Agency,CIA,CIA Central Intelligence Age
Federal,Other Federal Category,All Other Federal,5510,Federal Mediation & Conciliation Service,FMCS,Federal Mediation & Concilia
Federal,Other Federal Category,All Other Federal,5440,National Commission on Air Quality,nan,National Commission on Air Q
Federal,Other Federal Category,All Other Federal,5350,VA Department of Veterans Affairs,VA,VA Department of Veterans Af
Federal,Other Federal Category,All Other Federal,5325,Smithsonian - National Museum Natural History,nan,Smithsonian - National Museu
Federal,Other Federal Category,All Other Federal,5305,Smithsonian - Office of Fellowships & Grants,nan,Smithsonian - Office of Fell
Federal,Other Federal Category,All Other Federal,5302,Smithsonian - Office of Environmental Sciences,nan,Smithsonian - Office of Envi
Federal,Other Federal Category,All Other Federal,5301,Smithsonian Institution,nan,Smithsonian Institution
Federal,Other Federal Category,All Other Federal,5290,National Archives and Records Administration,nan,National Archives and Record
Federal,Other Federal Category,All Other Federal,5285,SBA Small Business Administration,SBA,SBA Small Business Administr


In [ ]:
combined_clean_name = combined_t.with_column('Direct Sponsor', 
                                        combined_t.apply(lambda x: x.split(".")[0], 'Direct Sponsor'))
combined_clean_name

Control Unit,Division,Department,Direct Sponsor,2014,2015,2016
Campus Support,Vice Chancellor - Administration,Finance,LBNL Lawrence Berkeley Nati,"$75,000","$400,000",nan
Campus Support,Vice Chancellor - Administration,Finance,SAMHSA Center for Substanc,$0,$0,nan
Campus Support,Vice Chancellor - Administration,Finance,UC Office of the President,$0,nan,nan
Campus Support,Vice Chancellor - Administration,Information Services & Technology,Andrew W Mellon Foundation,"$150,000",$0,nan
Campus Support,Vice Chancellor - Administration,Information Services & Technology,LYRASIS,nan,"$100,018","$70,854"
Campus Support,Vice Chancellor - Administration,Information Services & Technology,NSF National Science Founda,nan,nan,"$399,446"
Campus Support,Vice Chancellor - Administration,International House,Middlebury College,nan,"$10,000",nan
Campus Support,Vice Chancellor - Administration,Public Safety,Bay Areas Urban Area Securit,"$327,226","$30,145",nan
Campus Support,Vice Chancellor - Real Estate,Physical Plant Campus Services,California Emergency Manage,nan,nan,"$573,828"
Colleges,CNR College of Natural Resources,Null,"Abt Associates, Inc","$73,942",nan,nan


In [ ]:
# combined_clean_name.join("Direct Sponsor", phoebe_trunc, "SponsorName_trunc")
# 10/09 Note: Finding the corresponding "complete" names of sponsors
extended_names = {}
remnants = {}
for i in range(combined_clean_name.num_rows):
    source_name = combined_clean_name.column('Direct Sponsor')[i]
    target = phoebe_trunc.column("Sponsor")
    for j in range(len(target)):
        target_name = target[j]
        if (i) not in extended_names.keys() and source_name in target_name:
            extended_names[(i)] = target_name

In [ ]:
extended_names

In [ ]:
combined_t.column('Direct Sponsor')[991]

In [ ]:
orig = (np.array(list(extended_names.keys())))
missing = []
for elem in range(combined_clean_name.num_rows):
    if elem not in orig:
        missing += [elem]
print("Row IDs of sponsors in combined_clean_name not found", missing)

In [ ]:
combined_clean_name = combined_clean_name.with_column('index', range(combined_clean_name.num_rows))
combined_clean_name.show(3)

In [ ]:
combined_clean_name.row([1434, 2849])

In [ ]:
#adding the leftover 

# 10/09 Note: These two names are found in the corresponding rows of combined_clean_name but were either mispelled 
# or warped 
extended_names[1434] = "Helmholtz Zentrum Munchen"
extended_names[2849] = "Stifterverband fur die Deutsch"

In [ ]:
len(extended_names) 

## Starting with extended_names (as a means to join phoebe and data)

In [ ]:
extended_names_tbl = Table().with_columns(['index', extended_names.keys(),
                                           'Sponsor', extended_names.values()])
extended_names_tbl.show(2)

In [ ]:
combined_clean_name = combined_clean_name.join('index', extended_names_tbl)
combined_clean_name.show(100)

In [ ]:
phoebe.group('Sponsor').where('count', are.above(1))

In [ ]:
sponsor_to_class = {
    'Abt Associates, Inc.': 'Industry',
    'Academy of Sciences of the Czech Republic': 'Non Federal Governmental',
    'Intuitive Surgical, Inc.': 'Industry',
    'Nazarbayev University': 'Not for Profit',
    'Santa Clara Valley Water District': 'Non Federal Governmental',
    'Spanish National Research Council': 'Non Federal Governmental',
    'Twist Bioscience': 'Industry',
    'University of London': 'Not for Profit',
    'Van Strum Foundation': 'Not for Profit',
    'World Vision Inc.': 'Not for Profit',
    'New Profit, Inc.': 'Not for Profit',
    'Oak Ridge National Laboratory': 'Federal',
}

In [ ]:
def sponsor_to_class_mapper(row):
    if row.Sponsor in sponsor_to_class:
        return sponsor_to_class[row.Sponsor]
    else:
        return row['Sponsor Class']

In [ ]:
phoebe = phoebe.to_df().assign(class_clean = phoebe.to_df().apply(sponsor_to_class_mapper, axis=1))

In [ ]:
phoebe = phoebe.drop_duplicates(subset='Sponsor')

In [ ]:
phoebe = Table().from_df(phoebe)

In [ ]:
combined_phoebe = combined_clean_name.join('Sponsor', phoebe)
combined_phoebe.show(5)

In [ ]:
combined_phoebe_conc = combined_phoebe.select(['Sponsor', 'Control Unit', 'Division', 'Department', 'class_clean', '2014', '2015', '2016'])
combined_phoebe_conc.show(3)
# combined_phoebe.labels

In [ ]:
# 10/09 Note: Beginning to  clean dollar amounts
combined_phoebe_conc_pd = combined_phoebe_conc.to_df().fillna("Null")

In [ ]:
# dealing with parentheses and dollar signs 
def money_parse(strval):
    if strval == "Null" or strval == 'nan':
        return 0
    if "(" in strval:
        # Parentheses signify negatives 
        parsed = eval(strval.replace('($', '').replace(')', '').replace(',', '')) * -1
    else:
        parsed = eval(strval.replace('$', '').replace(',', ''))
    return parsed 

combined_phoebe_conc_pd['2014'] = combined_phoebe_conc_pd['2014'].apply(money_parse)
combined_phoebe_conc_pd['2015'] = combined_phoebe_conc_pd['2015'].apply(money_parse)
combined_phoebe_conc_pd['2016'] = combined_phoebe_conc_pd['2016'].apply(money_parse)

In [ ]:
combined_phoebe_conc_pd.head(3)

In [ ]:
# 10/09 Note: contains cleaned data and considers negatives 
combined_phoebe_conc_pd.to_csv('data/research_funding_clean.csv')

### Non-stem/stem

In [ ]:
# Divisions
stem = ['CNR College of Natural Resources', 'Chemistry Department', 'College of Chemistry',
       'College of Engineering', 'College of Environmental Design', 'Division of Physical Sciences',
       'L&S Biological Sciences', 'School of Information', 'School of Optometry', 
       ]
other = ['Strategic Academic and Facilities Planning', 'UC Library', 
        'Vice Provost Academic Affairs & Faculty Welfare', 'Vice Chancellor - Student Affairs Division',
        'Vice Chancellor - Real Estate', 'Vice Chancellor - Administration', 
        'VP Academic Planning & Fac', 'Office of Equity & Inclusion',
         'EVCP Executive Vice Chancellor & Provost']

In [ ]:
industry.show()

In [ ]:
# if division or department includes sciences, chemistry, engineering, transportation, 
def non_stem(div, dept):
    keywords = ['social sciences', 'international studies', 'education', 'labor and employment',
               'law', 'haas', 'regional development', 'humanities']
    div_ = np.any([x in div.lower() for x in keywords])
    dept_ = np.any([x in dept.lower() for x in keywords])
    return div_ or dept_

In [ ]:
stem_industry = industry.with_column('Non_Stem',
                                    industry.apply(non_stem, ['Division', 'Department']))
stem_industry.where('Non_Stem', True).num_rows

In [ ]:
non = sum(stem_industry.where('Non_Stem', True).column('Total'))
stem = sum(stem_industry.where('Non_Stem', False).column('Total'))
print(non, "to non-stem from industry")
print(stem, "to stem from industry")
print(stem/non ,": 1 stem to non-stem ratio")

### Other analysis on private funding

In [ ]:
industry.where("Total", are.below(0)).group('Department', sum).sort("Total sum", descending=False)
# Null is School of Optometry